# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Цель исследования
Построение модели машинного обучения, которая поможет определить регион, где добыча принесет наибольшую прибыль.

Исследование пройдет по следующим шагам:

1. Подготовка и обзор данных.
2. Обучение и проверка модели.
3. подготовка к расчету прибыли.
4. расчет прибыли и рисков.

Представлены следующие данные:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

Импортируем все необходимые библиотеки

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score

import warnings
#убираем системные предупреждения
warnings.filterwarnings('ignore')

Считаем данные из csv-файлов в датафрейм и сохраним в переменные data_one, data_two, data_three (данные геологоразведки трёх регионов находятся в трех файлах):

In [2]:
data_one = pd.read_csv('/datasets/geo_data_0.csv')
data_two = pd.read_csv('/datasets/geo_data_1.csv')
data_three = pd.read_csv('/datasets/geo_data_2.csv')

### Информация о данных первого региона

In [3]:
data_one.head() 

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
print(data_one.isna().sum())
print('\nЧисло дубликатов:', data_one.duplicated().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Число дубликатов: 0


In [5]:
data_one.info()
data_one.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
data_one.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [7]:
# удалим столбец id
data_one_del = data_one.drop(['id'], axis=1)

### Информация о данных второго региона

In [8]:
data_two.head() 

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
print(data_two.isna().sum())
print('\nЧисло дубликатов:', data_two.duplicated().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Число дубликатов: 0


In [10]:
data_two.info()
data_two.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
data_two.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [12]:
# удалим столбец id
data_two_del = data_two.drop(['id'], axis=1)

### Информация о данных третьего региона

In [13]:
data_three.head() 

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [14]:
print(data_three.isna().sum())
print('\nЧисло дубликатов:', data_three.duplicated().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Число дубликатов: 0


In [15]:
data_three.info()
data_three.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [16]:
data_three.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [17]:
# удалим столбец id
data_three_del = data_three.drop(['id'], axis=1)

**<u>Вывод по шагу 1:</u>**

Данные успешно загружены и не требуют дополнительной предобработки, пропуски и дубликаты отсутствуют, удален столбец id, чтобы избежать ошибок в дальнейшем.

Отметим так же, что на корреляции данных значений второго региона, мы видим сильную зависимость целевого показателя 'product' от f2. Значение коэффициента корреляции близко к 1, что означает, что есть сильная положительная связь между показателями - при увеличении одного показателя, значительно увеличивается и второй показатель.

Данные готовы к обучению.
Признаки:
- fo
- f1
- f2

Целевой признак:
- product

## Обучение и проверка модели

### Разобьём данные на обучающую и валидационную выборки в соотношении 75:25

In [18]:
# выделим признаки и целевой признак в каждом регионе
target_one=data_one_del['product']
features_one=data_one_del.drop(['product'], axis=1)

target_two=data_two_del['product']
features_two=data_two_del.drop(['product'], axis=1)

target_three=data_three_del['product']
features_three=data_three_del.drop(['product'], axis=1)

In [19]:
features_train_one, features_valid_one, target_train_one, target_valid_one = train_test_split(features_one, target_one, test_size=0.25, random_state=12345)

In [20]:
features_train_two, features_valid_two, target_train_two, target_valid_two = train_test_split(features_two, target_two, test_size=0.25, random_state=12345)

In [21]:
features_train_three, features_valid_three, target_train_three, target_valid_three = train_test_split(features_three, target_three, test_size=0.25, random_state=12345)

Данные трех регионов разбиты на обучающие и валидационные выборки в соотношении 75:25

### Обучим модель и сделаем предсказания на валидационной выборке.

In [22]:
# Создаем модель линейной регрессии для первого региона
model_one = LinearRegression()

In [23]:
# обучим модель на данных первого региона
model_one.fit(features_train_one, target_train_one)
one_predicted = model_one.predict(features_valid_one)
mse_one = mean_squared_error(target_valid_one, one_predicted)
rmse_one = mse_one**0.5
one_mean = one_predicted.mean()

print(f'Первый регион: средний запас предсказанного сырья: {one_mean}, RMSE = {rmse_one}')

Первый регион: средний запас предсказанного сырья: 92.59256778438035, RMSE = 37.5794217150813


In [24]:
# Создаем модель линейной регрессии для второго региона
model_two = LinearRegression()

In [25]:
# обучим модель на данных второго региона
model_two.fit(features_train_two, target_train_two)
two_predicted = model_two.predict(features_valid_two)
mse_two = mean_squared_error(target_valid_two, two_predicted)
rmse_two = mse_two**0.5
two_mean = two_predicted.mean()

print(f'Второй регион: средний запас предсказанного сырья: {two_mean}, RMSE = {rmse_two}')

Второй регион: средний запас предсказанного сырья: 68.728546895446, RMSE = 0.893099286775617


In [26]:
# Создаем модель линейной регрессии для третьего региона
model_three = LinearRegression()

In [27]:
# обучим модель на данных второго региона
model_three.fit(features_train_three, target_train_three)
three_predicted = model_three.predict(features_valid_three)
mse_three = mean_squared_error(target_valid_three, three_predicted)
rmse_three = mse_three**0.5
three_mean = three_predicted.mean()

print(f'Третий регион: средний запас предсказанного сырья: {three_mean}, RMSE = {rmse_three}')

Третий регион: средний запас предсказанного сырья: 94.96504596800489, RMSE = 40.02970873393434


**<u>Вывод по шагу 2:</u>**
1. разделили данные каждого региона на обучающую и валидационную выборки.
2. обучили модель линейной регрессии на тренировочных данных каждого региона.
3. посчитали среднее количество предсказанных запасов в каждом регионе и посчитали среднеквадратичную ошибку.
4. в первом регионе: средний запас предсказанного сырья: 92.59 тыс. баррелей, RMSE = 37.5794217150813.
5. во втором регионе: средний запас предсказанного сырья: 68.72 тыс. баррелей, RMSE = 0.893099286775617.
6. в третьем регионе: средний запас предсказанного сырья: 94.96 тыс. баррелей, RMSE = 40.02970873393434.
7. Лучше всего модель предсказала запасы во втором регионе (RMSE = 0.89), далее в тертьем и первом соответственно.

## Подготовка к расчёту прибыли

### Сохранение ключевых значений для расчётов в отдельных переменных

Условия задачи:

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [28]:
# сохраняем ключевые значения для регионов
best_mines_count = 200 # количество лучших скважин для разработки
budget = 10000000000 # Бюджет на разработку скважин в регионе
income_barrel_one = 450 # Доход с одного барреля сырья
income_unit = 450000 # Доход с каждой единицы продукта
probability_losses = 2.5/100 # вероятность убытков

###  Расчет достаточного объёма сырья для безубыточной разработки новой скважины

In [29]:
payback = budget/income_unit # рассчитываем количество единиц продукта, чтобы выйти в ноль при определнных вложениях (бюджете)
production_volume = payback/best_mines_count # объем сырья, 
round(production_volume,2)

111.11

### Сравнение полученного объёма сырья со средним запасом в каждом регионе. 

In [30]:
print('Cредний запас сырья в первом регионе: %.2f тыс.баррелей'%one_mean)
print('Cредний запас сырья во втором регионе: %.2f тыс.баррелей'%two_mean)
print('Cредний запас сырья в третьем регионе: %.2f тыс.баррелей'%three_mean)

Cредний запас сырья в первом регионе: 92.59 тыс.баррелей
Cредний запас сырья во втором регионе: 68.73 тыс.баррелей
Cредний запас сырья в третьем регионе: 94.97 тыс.баррелей


In [31]:
print('Разница объёма сырья по первому региону составляет: {:.2f} тыс.баррелей'.format(production_volume-one_mean))

Разница объёма сырья по первому региону составляет: 18.52 тыс.баррелей


In [32]:
print('Разница объёма сырья по второму региону составляет: {:.2f} тыс.баррелей'.format(production_volume-two_mean))

Разница объёма сырья по второму региону составляет: 42.38 тыс.баррелей


In [33]:
print('Разница объёма сырья по третьему региону составляет: {:.2f} тыс.баррелей'.format(production_volume-three_mean))

Разница объёма сырья по третьему региону составляет: 16.15 тыс.баррелей


Как видно, средний запас сырья во всех трех регионах ниже чем расчетное количество сырья для безубыточного производства.
Таким образом, в дальнейшем необходимо отбирать лучшие скважины.

### Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [34]:
# Выберем скважины с максимальными значениями предсказаний: посмотрим на значения первого региона.
one_predicted

array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229])

In [35]:
# отсортируем по убыванию
probs_sorted_one = pd.Series(one_predicted, index=target_valid_one.index).sort_values(ascending=False)
selected_max_one = probs_sorted_one[:200] # берем 200 лучших скважин по объему предсказанного сырья
selected_max_one

93073    180.180713
46784    176.252213
78948    175.850623
43388    175.658429
6496     173.299686
            ...    
30488    148.507064
98799    148.481767
53840    148.476498
4638     148.436761
75908    148.365941
Length: 200, dtype: float64

Так как затраты на разработку будут направлены только на 200 лучших скважин, сложим показатели скважин исходя из максимальных значений предсказаний нашей модели по объёму сырья для получения максимальной прибыли.

In [36]:
selected_max_one_sum = selected_max_one.sum()
print('Запас сырья в 200 лучших скважинах первого региона составит: %.2f тыс.баррелей'%selected_max_one_sum)

Запас сырья в 200 лучших скважинах первого региона составит: 31102.33 тыс.баррелей


In [37]:
# расчитаем прибыль для полученного объема сырья
profit_one = (selected_max_one_sum*income_unit) - budget
print('Прибыль для полученного объема сырья 200 лучших скважин первого региона: %.2f млн.рублей'%(profit_one/1000000))

Прибыль для полученного объема сырья 200 лучших скважин первого региона: 3996.05 млн.рублей


In [38]:
# внесем все эти формулы в функцию и посчитаем значения для трех регионов
def profit_best(target, predict,count):
    probs_sorted = pd.Series(predict, index=target.index).sort_values(ascending=False)
    selected = probs_sorted[:count].sum()
    profit_est=(selected*income_unit) - budget
    print('Запас сырья в 200 лучших скважинах составит: %.2f тыс.баррелей'%selected)
    print('Прибыль для полученного объема сырья 200 лучших скважин составит %.2f млн.рублей'%(profit_est/1000000))

In [39]:
# рассчитаем запас сырья и прибыль для первого региона
profit_best(target_valid_one, one_predicted, 200)

Запас сырья в 200 лучших скважинах составит: 31102.33 тыс.баррелей
Прибыль для полученного объема сырья 200 лучших скважин составит 3996.05 млн.рублей


In [40]:
# рассчитаем запас сырья и прибыль для второго региона
profit_best(target_valid_two, two_predicted, 200)

Запас сырья в 200 лучших скважинах составит: 27746.03 тыс.баррелей
Прибыль для полученного объема сырья 200 лучших скважин составит 2485.71 млн.рублей


In [41]:
# рассчитаем запас сырья и прибыль для третьего региона
profit_best(target_valid_three,three_predicted, 200)

Запас сырья в 200 лучших скважинах составит: 29603.90 тыс.баррелей
Прибыль для полученного объема сырья 200 лучших скважин составит 3321.75 млн.рублей


**<u>Вывод по шагу 3:</u>**
1. сохранили ключевые значения для расчетов в переменных.
2. сделали расчет необходимого количества сырья для безубыточной разработки новой скважины - это 111.11 тыс. баррелей;
3. выяснили, что среднее значение запасов во всех трех регионах меньше необходимого количества для безубыточной разработки
 - на 18,52 тыс.баррелей в первом регионе
 - на 42,38 тыс.баррелей во втором регионе
 - на 16,15 тыс.баррелей в третьем регионе
4. Ближе всего к необходимому количеству предсказанные запасы в третьем регионе, меньше всего предсказанных запасов во втором регионе.
5. прибыль для полученного объема сырья по регионам составит:
 - в первом регионе - 3996.05 млн.рублей
 - во втором регионе - 2485.71 млн.рублей
 - в третьем регионе - 3321.75 млн.рублей

Все три региона потенциально прибыльные, в следующей главе определим вероятность рисков убытков.

## Расчет прибыли и рисков для каждого региона

### Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [42]:
def profit_best_2(predict,target,count):
    probs_sorted = pd.Series(predict).sort_values(ascending=False)
    selected_count = target[probs_sorted.index][:count]
    selected_sum = selected_count.sum()
    revenue = selected_sum * income_unit
    return revenue - budget

In [43]:
def bootstrap_1000(target,predict):
    values = []
    count=0
    state = np.random.RandomState(12345)
    predict_a = pd.Series(predict, index=target.index)
    
    for i in range(1000):
        target_subsample = target.sample(n=500,replace=True, random_state=state)
  
        probs_subsample = predict_a[target_subsample.index]
        values.append(profit_best_2(probs_subsample,target_subsample, 200))  
    for i in values:
        if i < 0:
            count +=1
            
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    
     
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print("Средняя прибыль: {:.2f} млн.руб".format(mean/1000000))
    print('С вероятностью 95% прибыль в данном регионе будет составлять от: {:.2f} млн.руб до: {:.2f} млн.руб'.format((lower/1000000),(higher/1000000)))
    print("95%-ый доверительный интервал: от {:.2f} млн.руб до {:.2f} млн.руб". format((confidence_interval[0]/1000000),(confidence_interval[1]/1000000)))
    print ('Процент вероятности убытка в данном регионе составит {:.1%}'.format(count/1000))

### Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков

In [44]:
bootstrap_1000(target_valid_one,one_predicted)

Средняя прибыль: 425.94 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять от: -102.09 млн.руб до: 947.98 млн.руб
95%-ый доверительный интервал: от 408.73 млн.руб до 443.14 млн.руб
Процент вероятности убытка в данном регионе составит 6.0%


In [45]:
bootstrap_1000(target_valid_two,two_predicted)

Средняя прибыль: 515.22 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять от: 68.87 млн.руб до: 931.55 млн.руб
95%-ый доверительный интервал: от 501.62 млн.руб до 528.82 млн.руб
Процент вероятности убытка в данном регионе составит 1.0%


In [46]:
bootstrap_1000(target_valid_three,three_predicted)

Средняя прибыль: 435.01 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять от: -128.88 млн.руб до: 969.71 млн.руб
95%-ый доверительный интервал: от 417.45 млн.руб до 452.56 млн.руб
Процент вероятности убытка в данном регионе составит 6.4%


**<u>Вывод по шагу 4:</u>**

После применения техники Bootstrap, наиболее прибыльным будет разработка месторождений во втором регионе, так как в данном регионе самые высокие показатели средней выручки - 515.22 млн.руб и 95%-ного доверительного интервала - от 501.62 млн.руб до 528.82 млн.руб. Также во втором регионе самый низкий риск убытков - 1.0%.

# Общий вывод

Данные успешно загружены и проанализированы на предмет пропусков и дубликатов, удален столбец id, чтобы избежать ошибок в дальнейшем.
Признаки:

 - fo
 - f1
 - f2
 
Целевой признак:

 - product

На корреляции данных значений второго региона присутствует сильная зависимость целевого показателя 'product' от f2. Значение коэффициента корреляции близко к 1, что означает, что есть сильная положительная связь между показателями - при увеличении одного показателя, значительно увеличивается и второй показатель.

Далее:

 - разделили данные каждого региона на обучающую и валидационную выборки.
 - обучили модель линейной регрессии на тренировочных данных каждого региона.
 - посчитали среднее количество предсказанных запасов в каждом регионе и посчитали среднеквадратичную ошибку.
 - в первом регионе: средний запас предсказанного сырья: 92.59 тыс. баррелей, RMSE = 37.5794217150813.
 - во втором регионе: средний запас предсказанного сырья: 68.72 тыс. баррелей, RMSE = 0.893099286775617.
 - в третьем регионе: средний запас предсказанного сырья: 94.96 тыс. баррелей, RMSE = 40.02970873393434.
 
Лучше всего модель предсказала запасы во втором регионе (RMSE = 0.89), далее в тертьем и первом соответственно.

Ключевые значения для расчетов были сохранены в переменные.

По итогам расчета необходимого количества сырья для безубыточной разработки новой скважины у нас получилось - это 111.11 тыс. баррелей. Таким образом, среднее значение запасов во всех трех регионах меньше необходимого количества для безубыточной разработки во всех трех регионах:
 - на 18,52 тыс.баррелей в первом регионе
 - на 42,38 тыс.баррелей во втором регионе
 - на 16,15 тыс.баррелей в третьем регионе

Прибыль для полученного объема сырья по регионам составит:
 - в первом регионе - 3996.05 млн.рублей
 - во втором регионе - 2485.71 млн.рублей
 - в третьем регионе - 3321.75 млн.рублей

Все три региона потенциально прибыльные.

В связи с этим, ограничив вероятность убытка величиной 2,5% необходимо выбрать наиболее перспективный регион.
После применения техники Bootstrap, наиболее прибыльным будет разработка месторождений во втором регионе, так как в данном регионе самые высокие показатели средней выручки - 515.22 млн.руб и 95%-ного доверительного интервала - от 501.62 млн.руб до 528.82 млн.руб. Также во втором регионе самый низкий риск убытков - 1.0%. Второй регион рекомендован к разработке.